In [76]:
from src.data.dataset_functions import get_dataframe

dataFrame = get_dataframe(True)
dataFrame.show(1)

+--------------------+---+----+---+-----------+
|             user_id|  x|   y|  t|pixel_color|
+--------------------+---+----+---+-----------+
|vKtmwRQwAYtw5poO4...|804|1992|  0|    #FF3881|
+--------------------+---+----+---+-----------+
only showing top 1 row



<div style="color:pink">

Idee: 
-Daten nach Nutzern aggregieren und die Zeit zwischen aufeinanderfolgenden Pixeln vergleichen[erledigt] <br>
-Zeit (Differenz) festhalten und mit dem nächsten Zeitraum vergleichen [erledigt] <br>
-Falls die Differenz abweicht, ist der Nutzer wahrscheinlich kein Bot(?) <br>

</div>

In [ ]:
from src.features.feature_functions import group_dataframe_by_user
groupedFrame = group_dataframe_by_user(dataFrame)
#groupedFrame.show(5)
#brauchen wir gar nicht mehr :)

In [97]:
import pyspark.sql.functions as F
timeDiffFrame = dataFrame.alias('df1').join(dataFrame.alias('df2'),F.col('df1.user_id') == F.col('df2.user_id'),'inner')
timeDiffFrame = timeDiffFrame.where('df2.t > df1.t')
timeDiffFrame = timeDiffFrame.orderBy(['df1.user_id', 'df1.t'])
timeDiffFrame = timeDiffFrame.select(F.col('df1.user_id'),F.col('df1.t').alias('t1'),F.col('df2.t').alias('t2'),(F.col('df2.t') - F.col('df1.t')).alias('diff'))
timeDiffFrame.show(15)

+--------------------+----+----+----+
|             user_id|  t1|  t2|diff|
+--------------------+----+----+----+
|++/a8YPlilhzOTJEW...|1087|1427| 340|
|++/a8YPlilhzOTJEW...|1087|2995|1908|
|++/a8YPlilhzOTJEW...|1087|1427| 340|
|++/a8YPlilhzOTJEW...|1087|2995|1908|
|++/a8YPlilhzOTJEW...|1087|1427| 340|
|++/a8YPlilhzOTJEW...|1087|2995|1908|
|++/a8YPlilhzOTJEW...|1087|1427| 340|
|++/a8YPlilhzOTJEW...|1087|2995|1908|
|++/a8YPlilhzOTJEW...|1427|2995|1568|
|++/a8YPlilhzOTJEW...|1427|2995|1568|
|++/a8YPlilhzOTJEW...|1427|2995|1568|
|++/a8YPlilhzOTJEW...|1427|2995|1568|
|++/iKk3F3ZlK4sJxa...| 145| 548| 403|
|++/iKk3F3ZlK4sJxa...| 145| 548| 403|
|++/iKk3F3ZlK4sJxa...| 145| 870| 725|
+--------------------+----+----+----+
only showing top 15 rows



<div style ="color:pink">
-Differenz speichern und beim nächsten Vergleich erneut verwenden? [erledigt] <br>
-Nutzer, die das Kriterium bei einem der Vergleiche nicht erfüllen, müssten nicht erneut betrachtet werden(?) <br>
<br>
SELECT user_id, diff <br>
FROM timeDiffFrame <br>
WHERE NOT EXISTS(<br>
   SELECT *<br>
   FROM timeDiffFrame a, timeDiffFrame b<br>
   WHERE a.user_id = b.user_id<br>
   AND a.diff <> b.diff);<br>
</div>

In [98]:
timeDiffReduced = timeDiffFrame.dropDuplicates(['user_id', 't1'])
timeDiffReduced = timeDiffReduced.orderBy(['user_id', 't1'])
timeDiffReduced.show(15)

22/06/06 18:27:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 18:27:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 18:27:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 18:27:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 18:27:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 18:27:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 18:27:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 18:27:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 18:27:53 WARN RowBasedKeyValueBatch: Calling spill() on

+--------------------+----+----+----+
|             user_id|  t1|  t2|diff|
+--------------------+----+----+----+
|++/a8YPlilhzOTJEW...|1087|1427| 340|
|++/a8YPlilhzOTJEW...|1427|2995|1568|
|++/iKk3F3ZlK4sJxa...| 145| 548| 403|
|++/iKk3F3ZlK4sJxa...| 548| 870| 322|
|++/iKk3F3ZlK4sJxa...| 870|1201| 331|
|++/iKk3F3ZlK4sJxa...|1201|1553| 352|
|++/iKk3F3ZlK4sJxa...|1553|2174| 621|
|++/iKk3F3ZlK4sJxa...|2174|2487| 313|
|++/xB4v5Jeeacv4KO...|2037|2606| 569|
|++0X370V+dVFAmRQD...|2357|2660| 303|
|++0X370V+dVFAmRQD...|2660|2974| 314|
|++0X370V+dVFAmRQD...|2974|3291| 317|
|++0nCmngtZ68vbg6p...| 734|2690|1956|
|++0rQ+/WspcGyNaOH...|3008|3316| 308|
|++1HxWRIxcZdBaKrR...|1149|3310|2161|
+--------------------+----+----+----+
only showing top 15 rows



In [100]:
botDiff = timeDiffReduced.alias('tdr1').join(timeDiffReduced.alias('tdr2'),F.col('tdr1.user_id') == F.col('tdr2.user_id'),'inner')
botDiff = botDiff.where('tdr1.t1 <> tdr2.t1').where('tdr1.diff == tdr2.diff')
botDiff = botDiff.select('tdr1.user_id', 'tdr1.diff')
botDiff = botDiff.dropDuplicates(['user_id'])
botDiff = botDiff.orderBy(['user_id'])
botDiff.show(15)

22/06/06 18:29:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 18:29:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 18:29:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 18:29:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 18:29:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 18:29:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 18:29:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 18:29:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 18:29:53 WARN RowBasedKeyValueBatch: Calling spill() on

+--------------------+----+
|             user_id|diff|
+--------------------+----+
|++5ngkIIPwfxJtzv3...| 307|
|++6IlQ1s33JtGR24y...| 316|
|++ItjDxLoRLVe7/C4...| 315|
|++Lvwnim+QaifD9U4...| 307|
|++UiyqDibdpjwLi8O...| 307|
|++ZTAhuLp4CF8NxY2...| 329|
|++quEyKIDZltXAJO5...| 311|
|++t/2ZoXH2RQwnaVK...| 315|
|+/6XJYftUnMwTgGPl...| 316|
|+/8RPTGDQ4veBkUkY...| 311|
|+02eF0+7cl3Ol1b64...| 304|
|+05i4u0UwrZ9JkLMX...| 309|
|+0AL86CS7hWfXsH5s...| 312|
|+0PwSvdOri/m9qWYQ...| 307|
|+0RlYPM6KSjW8ShiI...| 303|
+--------------------+----+
only showing top 15 rows



In [101]:
print(botDiff.count())

22/06/06 18:30:12 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 18:30:12 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 18:30:12 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 18:30:12 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 18:30:12 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 18:30:12 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 18:30:12 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 18:30:12 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/06 18:30:12 WARN RowBasedKeyValueBatch: Calling spill() on

28205
